In [1]:
import cv2 
import numpy as np
import glob
import os
from itertools import combinations 
from itertools import product
import random


In [2]:
import keras 
from keras.layers import Dense, MaxPooling2D, Convolution2D, Dropout, Input, Flatten, Subtract, merge
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.models import Model

Using TensorFlow backend.


In [3]:
path_input_images =  '/tensorflow/IITD Database/'

In [4]:
folder_name_list = np.sort(os.listdir(path_input_images))
out1= np.zeros((8000,256,256,3),'float')
out2= np.zeros((8000,256,256,3),'float')

pos_comb1 = np.array(list(combinations([1, 2, 3, 4, 5], 2)))
pos_comb2 = np.array(list(combinations([6, 7, 8, 9, 10], 2)))

neg_comb1 = np.array(list(product([2,3,4,5], [8,9,10])))

count = 0
y = []
for folder_name in folder_name_list[:200]:
    
    for i in range (len(pos_comb1)):
        
        img1_0 = cv2.imread(path_input_images + folder_name + '/'+ str(pos_comb1[i][0]).zfill(2)+'.bmp')
        img1_0 = cv2.resize(img1_0,(256,256))
        img1_1 = cv2.imread(path_input_images + folder_name + '/'+ str(pos_comb1[i][1]).zfill(2)+'.bmp')
        img1_1 = cv2.resize(img1_1,(256,256))
        out1[count]  = img1_0
        out2[count]  = img1_1
        count = count+1
        y.append(1)
        
        img2_0 = cv2.imread(path_input_images + folder_name + '/'+ str(pos_comb2[i][0]).zfill(2)+'.bmp')
        img2_1 = cv2.imread(path_input_images + folder_name + '/'+ str(pos_comb2[i][1]).zfill(2)+'.bmp')
        img2_0 = cv2.resize(img2_0,(256,256))
        img2_1 = cv2.resize(img2_1,(256,256))
        out1[count]  = img2_0
        out2[count]  = img2_1
        y.append(1)
        count = count+1
        
        img3_0 = cv2.imread(path_input_images + folder_name + '/'+ str(neg_comb1[i][0]).zfill(2)+'.bmp')
        img3_1 = cv2.imread(path_input_images + folder_name + '/'+ str(neg_comb1[i][1]).zfill(2)+'.bmp') 
        
        img3_0 = cv2.resize(img3_0,(256,256))
        img3_1 = cv2.resize(img3_1,(256,256))
        out1[count]  = img3_0
        out2[count]  = img3_1
        y.append(0)
        count = count+1
        
        folder_name1 = str(int(folder_name) + 3).zfill(3) 
        
        img4_0 = cv2.imread(path_input_images + folder_name + '/'+ str(pos_comb1[i][0]).zfill(2)+'.bmp')
        img4_1 = cv2.imread(path_input_images + folder_name1 + '/'+ str(pos_comb1[i][1]).zfill(2)+'.bmp') 
        img4_0 = cv2.resize(img4_0,(256,256))
        img4_1 = cv2.resize(img4_1,(256,256))
        out1[count]  = img4_0
        out2[count]  = img4_1
        y.append(0)
        count = count+1
        #print(count) 

In [5]:

out1 = out1/255.
out2 = out2/255.

MemoryError: 

In [ ]:
print ("X_train shape: " + str(out1_train.shape))


In [ ]:
path_input_images + folder_name + str(pos_comb1[i][0]).zfill(2)+'.bmp'

In [ ]:
def Siamese_model(input_shape):    
    inp_1 = Input(shape=input_shape)
    conv_1 = Convolution2D(64,(10,10),activation='relu')(inp_1)
    max_2 = MaxPooling2D((2,2),strides=(2,2))(conv_1)
    conv_3 = Convolution2D(128,(7,7),activation='relu')(max_2)
    max_4 = MaxPooling2D((2,2),strides=(2,2))(conv_3)
    conv_5 = Convolution2D(128,(4,4),activation='relu')(max_4)
    max_6 = MaxPooling2D((2,2),strides=(2,2))(conv_5)
    conv_7 = Convolution2D(256,(4,4),activation='relu')(max_6)
    fcl = Flatten()(conv_7)
    dense_1 = Dense(4096,activation='relu')(fcl)

    inp_2 = Input(shape=input_shape)
    conv2_1 = Convolution2D(64,(10,10),activation='relu')(inp_2)
    max2_2 = MaxPooling2D((2,2),strides=(2,2))(conv2_1)
    conv2_3 = Convolution2D(128,(7,7),activation='relu')(max2_2)
    max2_4 = MaxPooling2D((2,2),strides=(2,2))(conv2_3)
    conv2_5 = Convolution2D(128,(4,4),activation='relu')(max2_4)
    max2_6 = MaxPooling2D((2,2),strides=(2,2))(conv2_5)
    conv2_7 = Convolution2D(256,(4,4),activation='relu')(max2_6)
    fcl2 = Flatten()(conv2_7)
    dense2_1 = Dense(4096,activation='relu')(fcl2)

    L1_distance = lambda x: K.abs(x[0]-x[1])
    dense_layer = merge([dense_1,dense2_1], mode = L1_distance, output_shape=lambda x: x[0])

    out = Dense(1,activation='softmax')(dense_layer)
    
    model = Model(inputs=[inp_1,inp_2],outputs=out,name='Siamese_model')
    return model

In [ ]:
siamese_model = Siamese_model((256,256,3))

In [ ]:
siamese_model.summary()

In [ ]:
siamese_model.compile(loss='binary_crossentropy', optimizer="Adam", metrics = ["accuracy"])

In [ ]:
siamese_model.fit([out1,out2],y,epochs=10, batch_size = 4)